In [1]:
import csv
import exrex # !pip install exrex
import pandas as pd
import numpy as np
import re 
import spacy

from tqdm import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

# ebablbe auto-completion
%config Completer.use_jedi = False

## Generate Config File

download model.

In [2]:
# !python -m spacy download en_core_web_trf

generate base_config.cfg and base_config_gpu.cfg on https://spacy.io/usage/training#quickstart

In [3]:
# !python -m spacy init fill-config base_config.cfg config.cfg
# !python -m spacy init fill-config base_config_gpu.cfg config_gpu.cfg

## Gather Data

###  Convert Excel to Txt
upload the excel file to http://15.15.166.35:18888/tree/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx

~~~shell
!pip install xlrd==1.2.0
~~~

In [4]:
# !ls -l /tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/xlsx/ 

In [5]:
# ! /tf/eipi10/jian-xu3/snps-classification/snps/etl/xlsx2txt.py  case_note_ner '' 1 '\t'

### Load Txt File

In [6]:

def load_data(text_file):
    names = ['Case_ID', 'Work_Order_Number', 'Created_On', 'Service_Account', 'Due_Date', 
             'Action_Booking_status', 'Action_Date', 'Currently_Worked_by', 'Owner', 'Product_Line', 
             'Follow_Up_Reason_Code', 'Response_Time', 'Work_Order_Type', 'Postal_Code', 'Active_Booking', 
             'Follow_Up_Type', 'Business_Segment', 'Follow_Up_Note', 'Reschedule_Note']
    dtypes = [np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str, 
              np.str, np.str, np.str, np.str, np.str,
              np.str, np.str, np.str, np.str]
    dtypes = {name:dtype for name, dtype in zip(names, dtypes)}
    df = pd.read_table(text_file, names=names, dtype=dtypes, quoting=csv.QUOTE_NONE)

    df.index = range(1, len(df)+1) 
    df['id'] = df.index
      
    return df

text_file='/tf/eipi10/jian-xu3/snps-classification/snps/etl/data/case_note_ner/txt/3RS_July_Extraction_with_notes_004.txt'
df_case = load_data(text_file)

In [7]:
print('-'*50)
print(len(df_case))

print('-'*50)
display(df_case.head(5))

print('-'*50)
display(df_case.describe().T)

print('-'*50)
df_case.info()



--------------------------------------------------
7039
--------------------------------------------------


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,Follow_Up_Reason_Code,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,NaN,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,NaN,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4
5,5061278521,WO-013010632-4,2021-06-16 12:17:00,MR. REGINALD JOSEPH,2021-07-15 14:42:00,Reschedule,2021-07-15 13:09:00,NaN,JOCILYN ANDREA SANDI DURAN,Commercial Premium Notebooks,NaN,NCD,Break Fix,20785,USA-PR-USYWWDWN_E_A_EEG,Reschedule,Computing,NaN,NaN,5


--------------------------------------------------


,count,mean,std,min,25%,50%,75%,max
id,7039.0,3520.0,2032.128605,1.0,1760.5,3520.0,5279.5,7039.0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7039 entries, 1 to 7039
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Case_ID                7039 non-null   object
 1   Work_Order_Number      7039 non-null   object
 2   Created_On             7039 non-null   object
 3   Service_Account        7038 non-null   object
 4   Due_Date               7017 non-null   object
 5   Action_Booking_status  4223 non-null   object
 6   Action_Date            3408 non-null   object
 7   Currently_Worked_by    887 non-null    object
 8   Owner                  7039 non-null   object
 9   Product_Line           6457 non-null   object
 10  Follow_Up_Reason_Code  2864 non-null   object
 11  Response_Time          7039 non-null   object
 12  Work_Order_Type        7039 non-null   object
 13  Postal_Code            7039 non-null   object
 14  Active_Booking       

In [8]:
print(df_case.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1666
Onsite Stock Replenishment     376
Partner Request Reassign        16
Request Reassign              2100
Reschedule                     434
Revisit                       2447
dtype: int64


### Data Cleaning

In [9]:
def filter_out_note_na(df):
    df_filter = df.loc[(df.Follow_Up_Note.notna()) | (df.Reschedule_Note.notna())].copy()
    return df_filter

df_train_test = filter_out_note_na(df_case)
print(len(df_train_test))

# df_train_test['notes'] = [('' if pd.isna(fun) else fun)  + ('' if pd.isna(crn) else ' | ' + crn)
#                           for fun, crn in zip(df_train_test.Follow_Up_Note, df_train_test.Reschedule_Note)] 
df_train_test['notes'] = [(crn if pd.isna(fun) else (fun if pd.isna(crn) else fun + '.' + crn))
                          for fun, crn in zip(df_train_test.Follow_Up_Note, df_train_test.Reschedule_Note)] 
df_train_test

6110


,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
1,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,#NORMAL Part escalated,1,#NORMAL Part escalated
2,5060815932,WO-012952625,2021-02-03 10:38:00,William Torres,2021-04-26 15:00:00,Additional Parts Required,2021-07-03 11:45:00,Kerby Singleton,Nancy Quesada,Commercial Desktop PC Premium L6,...,NCD,Break Fix,30305,PC-Kerby Singleton,Additional Parts Required,Computing,NaN,Please cancel backordered part. Customer has c...,2,Please cancel backordered part. Customer has c...
3,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,According to bryan this laptop shows repair co...,3,According to bryan this laptop shows repair co...
4,5061272897,WO-013009621,2021-02-09 13:39:00,LAS VEGAS METRO POLICE DEPT,2021-02-24 19:00:00,Additional Parts Required,2021-07-16 15:30:00,NaN,Avinash Shukla,Commercial Premium Notebooks,...,NCD,Break Fix,89101,PC-Joe Walper,Additional Parts Required,Computing,NaN,Back order part,4,Back order part
6,5064053283,WO-013536670-2,2021-06-30 11:06:00,Ford Motor Company of Canada,2021-07-02 08:30:00,Request Reassign,2021-07-05 14:45:00,NaN,Gabriel Bustos,A3 Laserjet,...,NCD,IMACD,N9A,Rick Harbarenko,Request Reassign,dMPS,NaN,Please send this case to Corey Harbarenko. Cor...,6,Please send this case to Corey Harbarenko. Cor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,5069292833,WO-014560602,2021-07-16 16:23:00,Kaiser Foundation Health Plan Inc.,2021-07-19 15:09:00,Additional Parts Required,2021-07-16 16:00:00,NaN,6 CSPAE166,A4 PageWide SMB & Enterprise managed,...,Not available,Break Fix,91786,Anthony Garcia,Additional Parts Required,pMPS,NaN,Please order the following part. Qty. 1 B5L04...,7035,Please order the following part. Qty. 1 B5L04...
7036,5069293232,WO-014560595,2021-07-16 16:21:00,CDW - St. Lukes Hospital (Government),2021-07-20 16:26:00,Request Reassign,NaN,NaN,1 CSPAE131,A4 Laserjet SMB,...,Not available,Break Fix,55805,Brock Tharp,Request Reassign,pMPS,NaN,Please reassign to lms,7036,Please reassign to lms
7037,5069293974,WO-014560699,2021-07-16 17:06:00,LITHIA NISSAN OF FRESNO,2021-07-19 18:00:00,Request Reassign,2021-07-19 13:00:00,Aaron Stavinsky,Luis Alejandro Vargas,A4 Laserjet Enterprise,...,NCD,Preventative Maintenance,93710,IS-Aaron Stavinsky,Request Reassign,Printing,NaN,dahlia.garcia@hp.com,7037,dahlia.garcia@hp.com
7038,5069294340,WO-014560697,2021-07-16 17:04:00,"Best Buy co., Inc.",2021-07-20 18:00:00,NaN,NaN,NaN,Subir Das,Retail Solutions Core,...,NCD,Break Fix,95678,PR-David Popadiuc,Revisit,Computing,Please order 842275-001 and have it ship to HF...,NaN,7038,Please order 842275-001 and have it ship to HF...


In [10]:
print(df_train_test.groupby(['Follow_Up_Type']).size())

Follow_Up_Type
Additional Parts Required     1579
Onsite Stock Replenishment     375
Partner Request Reassign        13
Request Reassign              1737
Reschedule                     136
Revisit                       2270
dtype: int64


### Generate NER Label

In [11]:
def standardize_text1(text):
    text = text.strip().lower()
    # add space between punctuation
    text = re.sub(r'([.\\!?,\'()\[\]"|;])', r' \1 ', text)
#     # remove characters except for English letters and some punctuations
#     text = re.sub(r"[^A-Za-z\.\-\?\!\,\#\@\% ]", "", text)
    # remove extra spaces
    text = re.sub(r'[" "]+', " ", text)
    return text

In [12]:
nlp = spacy.load("en_core_web_sm")
notes = [standardize_text1(note) for note in df_train_test.notes]
docs = nlp.pipe(notes)

In [13]:
def get_samples(df):
    df_filter = df.loc[(df.Work_Order_Number=='WO-013865844-1') | 
                       (df.Work_Order_Number=='WO-013910857') |
                       (df.Work_Order_Number=='WO-014466680') |
                       (df.Work_Order_Number=='WO-014466875') |
                       (df.Work_Order_Number=='WO-014466875-1') 
                      ].copy()
    return df_filter

df_sample = get_samples(df_train_test)
df_sample

,Case_ID,Work_Order_Number,Created_On,Service_Account,Due_Date,Action_Booking_status,Action_Date,Currently_Worked_by,Owner,Product_Line,...,Response_Time,Work_Order_Type,Postal_Code,Active_Booking,Follow_Up_Type,Business_Segment,Follow_Up_Note,Reschedule_Note,id,notes
21,5065852054,WO-013865844-1,2021-07-02 14:10:00,KAISER PERMANENTE,2021-07-16 18:00:00,NaN,NaN,Ryan Bartoli,Ezequiel Jara Corao,Detachables,...,NCD,Break Fix,94553,CT-Ryan Bartoli,Revisit,Computing,Please create a revisit for Wednesday,#NORMAL Please order part # L51580-001 and L31...,21,Please create a revisit for Wednesday.#NORMAL ...
29,5066093878,WO-013910857,2021-05-14 17:16:00,AMD,2021-07-02 17:00:00,NaN,NaN,NaN,JORGE DANIEL BENAVIDEZ AVILA,Commercial Premium Notebooks,...,NCD,Break Fix,78735,PR-Rob Silcocks,Revisit,Computing,Please order battery 933321-855 ship to UPS H...,Please reassign to Rob Silcocks,29,Please order battery 933321-855 ship to UPS H...
2978,5068832635,WO-014466680,2021-07-08 10:24:00,Jabil circuit Inc,2021-07-09 17:00:00,Additional Parts Required,2021-07-15 08:00:00,NaN,7 CSPAE147,Large Format Design Pagewide XL HW,...,NCD,Break Fix,87106,PR-Isaac Lucero,Additional Parts Required,dMPS,NaN,"Per L2 please order: 2YB64-67005, 2YB64-67003 ...",2978,"Per L2 please order: 2YB64-67005, 2YB64-67003 ..."
3025,5068836700,WO-014466875,2021-07-08 10:49:00,Border Patrol and Customs,2021-07-14 16:00:00,NaN,NaN,NaN,Abu Ambia,Large Format Design HW,...,NCD,Break Fix,79925,PR-Manuel Bailon,Revisit,Printing,Please create a revisite for 7-16-2021 at 1300...,NaN,3025,Please create a revisite for 7-16-2021 at 1300...
3026,5068836700,WO-014466875-1,2021-07-14 15:01:00,Border Patrol and Customs,2021-07-16 16:00:00,NaN,NaN,NaN,Lisbeth Alejandra Sibaja,Large Format Design HW,...,NCD,Break Fix,79925,PR-Manuel Bailon,Revisit,Printing,Please create a revisite for 7-23-2021 for pri...,NaN,3026,Please create a revisite for 7-23-2021 for pri...


In [14]:

sample_notes = [standardize_text1(note) for note in df_sample.notes]
sampel_docs = nlp.pipe(sample_notes)
for i, doc in enumerate(sampel_docs):
    print('-'*50)
    print(doc.text)
    print([(token.text, token.pos_) for token in doc])
    print([(ent.text, ent.label_) for ent in doc.ents]) 

--------------------------------------------------
please create a revisit for wednesday . #normal please order part # l51580-001 and l31365-001 to concord , california hal setting up time to meet with customer . 
[('please', 'INTJ'), ('create', 'VERB'), ('a', 'DET'), ('revisit', 'NOUN'), ('for', 'ADP'), ('wednesday', 'PROPN'), ('.', 'PUNCT'), ('#', 'NOUN'), ('normal', 'ADJ'), ('please', 'INTJ'), ('order', 'VERB'), ('part', 'NOUN'), ('#', 'SYM'), ('l51580', 'PROPN'), ('-', 'PUNCT'), ('001', 'NUM'), ('and', 'CCONJ'), ('l31365', 'NOUN'), ('-', 'PUNCT'), ('001', 'NUM'), ('to', 'ADP'), ('concord', 'PROPN'), (',', 'PUNCT'), ('california', 'PROPN'), ('hal', 'NOUN'), ('setting', 'VERB'), ('up', 'ADP'), ('time', 'NOUN'), ('to', 'PART'), ('meet', 'VERB'), ('with', 'ADP'), ('customer', 'NOUN'), ('.', 'PUNCT')]
[('wednesday', 'DATE'), ('# l51580-001', 'MONEY'), ('concord', 'GPE'), ('california', 'GPE')]
--------------------------------------------------
please order battery 933321-855 ship to ups

In [176]:
patterns = [
 {'expression': r'[a-zA-Z]\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d\d-[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d-\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]-\d\d\d\d-\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d-[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d[a-zA-Z]-\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z][a-zA-Z]\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d-\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d\d-\d\d-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-[a-zA-Z]\d[a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d[a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d[a-zA-Z]\d-\d\d\d\d-\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d[a-zA-Z]\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d-\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d[a-zA-Z][a-zA-Z]-\d\d\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-\d\d\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d-\d\d-\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d\d[a-zA-Z]\d[a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d\d[a-zA-Z]\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z][a-zA-Z]-\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d-\d\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\d[a-zA-Z]\d\d[a-zA-Z]\d-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d\d\d\d-\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d-\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d-\d\d\d\d-\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z]\d[a-zA-Z]\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d\d\d-[a-zA-Z]\d\d\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d-[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d-[a-zA-Z]\d', 'label': 'PART_NUM'}, 
 {'expression': r'\d\d\d\d\d\d-\d\d\d[a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d\d-\d\d\d\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z]-\d\d[a-zA-Z]\d[a-zA-Z]\d\d', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z]\d\d[a-zA-Z]-[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
 {'expression': r'[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]-\d\d\d\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}, 
#  {'expression': r'order', 'label': 'ACTION'}, 
#  {'expression': r'recommended', 'label': 'ACTION'}, 
#  {'expression': r'hfpu', 'label': 'ACTION'}, 
#  {'expression': r'ship', 'label': 'ACTION'}, 
#  {'expression': r'fedex', 'label': 'ACTION'}, 
#  {'expression': r'hal', 'label': 'ACTION'}, 
#  {'expression': r'site', 'label': 'ACTION'}, 
#  {'expression': r'send', 'label': 'ACTION'}, 
#  {'expression': r'location', 'label': 'ACTION'}, 
#  {'expression': r'pickup', 'label': 'ACTION'}, 
#  {'expression': r'sent', 'label': 'ACTION'}, 
#  {'expression': r'requested', 'label': 'ACTION'}, 
#  {'expression': r'reorder', 'label': 'ACTION'}, 
#  {'expression': r'street', 'label': 'ACTION'}, 
#  {'expression': r'hfp', 'label': 'ACTION'}, 
#  {'expression': r'shipped', 'label': 'ACTION'}, 
#  {'expression': r'delivery', 'label': 'ACTION'}, 
#  {'expression': r'recommend', 'label': 'ACTION'}, 
#  {'expression': r'warehouse', 'label': 'ACTION'}    
]

# remove duplidate data
patterns1 = {}
for i, pattern in enumerate(patterns):
    expression = pattern['expression']
    patterns1[expression] = pattern
    
patterns = [value for _, value in patterns1.items()]
print(len(patterns))
print(*patterns, sep="\n") 

138
{'expression': '[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 'label': 'PART_NUM'}
{'expression': '\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 'label': 'PART_NUM'}
{'expression': '[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 'label': 'PART_NUM'}
{'expression': '\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 'label': 'PART_NUM'}
{'expression': '\\d\\d\\d\\d\\d\\d-\\d[a-zA-Z]\\d', 'label': 'PART_NUM'}
{'expression': '\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}
{'expression': '[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 'label': 'PART_NUM'}
{'expression': '\\d\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 'label': 'PART_NUM'}
{'expression': '[a-zA-Z][a-zA-Z]-\\d\\d\\d\\d\\d-\\d\\d', 'label': 'PART_NUM'}
{'expression': '[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 'label': 'PART_NUM'}
{'expression': '\\d\\d\\d\\d-\\d\\d\\d\\d', 'label': 'PART_NUM'}
{'expression': '\\d\\d\\d\\d-\\d\\d\\d\\d-\\d', 'label': 'PART_NUM'}
{'expression': '\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 'label': 'PART_N

In [177]:
common_words_patterns = [
 {'expression': r'order', 'label': 'ACTION'}, 
 {'expression': r'recommended', 'label': 'ACTION'}, 
 {'expression': r'hfpu', 'label': 'ACTION'}, 
 {'expression': r'ship', 'label': 'ACTION'}, 
 {'expression': r'fedex', 'label': 'ACTION'}, 
 {'expression': r'hal', 'label': 'ACTION'}, 
 {'expression': r'site', 'label': 'ACTION'}, 
 {'expression': r'send', 'label': 'ACTION'}, 
 {'expression': r'location', 'label': 'ACTION'}, 
 {'expression': r'pickup', 'label': 'ACTION'}, 
 {'expression': r'sent', 'label': 'ACTION'}, 
 {'expression': r'requested', 'label': 'ACTION'}, 
 {'expression': r'reorder', 'label': 'ACTION'}, 
 {'expression': r'street', 'label': 'ACTION'}, 
 {'expression': r'hfp', 'label': 'ACTION'}, 
 {'expression': r'shipped', 'label': 'ACTION'}, 
 {'expression': r'delivery', 'label': 'ACTION'}, 
 {'expression': r'recommend', 'label': 'ACTION'}, 
 {'expression': r'warehouse', 'label': 'ACTION'}   
]

In [181]:
def generate_label(texts, patterns={}):   
    
    def entity_from_patterns(text, patterns, pattern_matchs):        
        entities = {}
#         print('-'*50)  
        for pattern in patterns:
            entities = entity_from_pattern(text, pattern, entities, pattern_matchs)
        entities = [value for _, value in sorted(entities.items(), key=lambda item: item[0][0])]
        return entities

    def entity_from_pattern(text, pattern, entities, pattern_matchs):
        expression = pattern['expression']
        label = pattern['label']
        for match in re.finditer(expression, text):
            start, end = match.span()    
            add_key = True
            if start-1 >= 0 and text[start-1]!=' ':
                add_key = False
            if end < len(text) and text[end]!=' ':
                add_key = False                
                
#             drop_keys = []
#             for key, value in entities.items():
#                 start_, end_ = key
#                 if start<start_ and end>end_:
#                     drop_keys.append(key)
#                 if start>start_ and end<end_:
#                     add_key = False
#             for key in drop_keys:
#                 entities.pop(key)

            if add_key: 
#                 print(text, expression)
                pattern_matchs[expression] = pattern_matchs[expression] + 1
                entities[(start, end)] = (start, end, label)
        return entities
        
    examples = []
    pattern_matchs = {pattern['expression']:0 for pattern in patterns}
    for text in texts:
        entities = entity_from_patterns(text, patterns, pattern_matchs)        
        example = (text, {"entities": entities})
        examples.append(example)
    pattern_matchs = [(key, value) for key, value in sorted(pattern_matchs.items(), key=lambda item: -item[1])]
#     pattern_matchs = [(key, value) for key, value in pattern_matchs if value>0]
        
    return examples, pattern_matchs

see some sample data.

In [183]:
def show_entity_label(text_entities, pattern_matchs):
    for example in text_entities:
        print('-'*50)
        text, annotations = example
        print(text) 
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')
    print('='*50)
    print(*pattern_matchs, sep='\n')
    
texts = [standardize_text1(note) for note in df_sample.notes]  
sample_text_entities, sample_pattern_matchs = generate_label(texts, patterns=patterns)
show_entity_label(sample_text_entities, sample_pattern_matchs)

--------------------------------------------------
please create a revisit for wednesday . #normal please order part # l51580-001 and l31365-001 to concord , california hal setting up time to meet with customer . 
('l51580-001', 68, 78, 'PART_NUM')
('l31365-001', 83, 93, 'PART_NUM')
--------------------------------------------------
please order battery 933321-855 ship to ups hold for pickup , tuscany way , austin . . please reassign to rob silcocks
('933321-855', 21, 31, 'PART_NUM')
--------------------------------------------------
per l2 please order: 2yb64-67005 , 2yb64-67003 xl3600 , 6kd23-67021 xl3600 , 6kd23-67030 , and two 2yb64-67002
('2yb64-67005', 21, 32, 'PART_NUM')
('2yb64-67003', 35, 46, 'PART_NUM')
('6kd23-67021', 56, 67, 'PART_NUM')
('6kd23-67030', 77, 88, 'PART_NUM')
('2yb64-67002', 99, 110, 'PART_NUM')
--------------------------------------------------
please create a revisite for 7-16-2021 at 1300 order part number q5669-60687 please order 2 q6683-67001 t8w18-67001 s

In [184]:
texts = [standardize_text1(note) for note in df_train_test.notes] 
text_entities, pattern_matchs = generate_label(texts, patterns=patterns)
print(len(text_entities))
print(*pattern_matchs, sep='\n')

6110
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 1089)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 643)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 547)
('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 483)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 400)
('[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 271)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d\\d\\d\\d', 199)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 122)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 83)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 39)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z]\\d\\d', 23)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]\\d\\d[a-zA-Z]', 21)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 7)
('\\d\\d-\\d\\d\\d\\d\\d-\\d\\d\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d[a-zA-Z]\\d[a-zA-Z][a-zA-Z]\\d', 5)
('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d[a-zA-Z]', 5)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 3)
('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z]\\

In [185]:
print(np.sum([len(annotations['entities']) for text, annotations in text_entities]))
np.sum([value for key, value in pattern_matchs]) 

3956


3956

In [201]:
train_entities, test_entiteis = train_test_split(text_entities, test_size=0.25, random_state=199)

print(len(train_entities))
print(len(test_entiteis))

4582
1528


### Data Augmentation

In [187]:
expression = '[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d-\\d\\d\\d\\d'
exrex.getone(expression).lower()

'kcf-943-4087'

In [188]:
def get_augument_rates(pattern_counts):
    pattern_counts = np.array([300+np.sqrt(count-300) if count>300 else count for count in pattern_counts ] )
    pattern_counts = max(pattern_counts) - pattern_counts
    augument_rates = pattern_counts/np.sum(pattern_counts)
    
    return augument_rates

pattern_matchs_dict = {key:value for key, value in pattern_matchs}   
part_num_patterns = [pattern for pattern in patterns if pattern['label']=='PART_NUM' ]
part_num_pattern_counts = [pattern_matchs_dict.get(pattern['expression'], 0) for pattern in part_num_patterns]

part_num_augument_rates = get_augument_rates(part_num_pattern_counts) 
print(max(part_num_augument_rates), sum(part_num_augument_rates))
[(cnt, rate) for cnt, rate in zip(part_num_pattern_counts, part_num_augument_rates)]

0.007648398774009697 0.9999999999999994


[(547, 0.0002884367030158533),
 (400, 0.00042169327438072816),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (3, 0.007578462914335869),
 (0, 0.007648398774009697),
 (2, 0.007601774867560479),
 (643, 0.00022306939097441325),
 (83, 0.005713506656367102),
 (0, 0.007648398774009697),
 (39, 0.006739232598249924),
 (1089, 0.0),
 (0, 0.007648398774009697),
 (122, 0.004804340480607329),
 (0, 0.007648398774009697),
 (483, 0.0003394545486791547),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (5, 0.00753183900788665),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.007648398774009697),
 (0, 0.0076483987

In [202]:
def augment(data_entities, patterns, augument_rates, augument_num=40000):
    i = 0
    new_texts = []
    
    augment_matches = {}
    while i<augument_num:
        data_index = int(np.random.randint(len(data_entities), size=1))
        text, annotations = data_entities[data_index]
        entities = [(start, end, label) for start, end, label in annotations['entities'] if label=='PART_NUM']        
        if len(entities)<=0:
            continue
        entities.reverse()
        new_text = text
        for start, end, label in entities:                       
            pattern_index = int(np.random.choice(a=len(augument_rates), size=1, replace=True, p=augument_rates))
            pattern = patterns[pattern_index]['expression']
            part_num = exrex.getone(pattern).lower()
            new_text = new_text[0:start] + part_num + new_text[end:] 
            augment_matches[pattern] = augment_matches.get(pattern, 0) + 1
        new_texts.append(new_text)   

        i = i + 1
        if i % 5000==0:
            print(f'{i}/{augument_num}')
    augment_matches = [(key, value) for key, value in sorted(augment_matches.items(), key=lambda item: -item[1])]
    return new_texts, augment_matches
        
    
sample_augment_texts, sample_augment_matches =  augment(tran_text_entities[0:10], patterns,       
                                                        part_num_augument_rates, augument_num=10)
sample_augment_matches

[('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d\\d\\d\\d\\d\\d\\d', 2),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]\\d\\d[a-zA-Z]-[a-zA-Z]',
  1),
 ('\\d\\d\\d\\d-\\d\\d\\d\\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d[a-zA-Z]', 1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]',
  1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d', 1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d\\d[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d',
  1),
 ('[a-zA-Z]\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d[a-zA-Z]-\\d\\d\\d\\d\\d', 1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d-\\d\\d\\d\\d\\d-\\d\\d\\d', 1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\\d[a-zA-Z]\\d\\d[a-zA-Z]\\d-\\d\\d',
  1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-[a-zA-Z]\\d\\d\\d-[a-zA-Z]\\d', 1),
 ('[a-zA-Z][a-zA-Z][a-zA-Z]-\\d\\d\\d[a-zA-Z][a-zA-Z]\\d\\d\\d\\d\\d', 1),
 ('

In [209]:
tran_augment_texts, train_augment_matches = augment(tran_text_entities, patterns, 
                                                    part_num_augument_rates, augument_num=9000)
print(len(tran_augment_texts))
train_augment_text_entities, train_augment_pattern_matchs = generate_label(tran_augment_texts, patterns=patterns)
print(np.sum([value for key, value in train_augment_pattern_matchs]))
print(np.sum([len(annotations['entities']) for text, annotations in train_augment_text_entities]))

5000/9000
9000
13443
13443


In [213]:
test_augment_texts, test_augment_matches = augment(test_text_entities, patterns, 
                                                   part_num_augument_rates, augument_num=3000)
print(len(test_augment_texts))
test_augment_text_entities, test_augment_pattern_matchs = generate_label(test_augment_texts, patterns=patterns)
print(np.sum([value for key, value in test_augment_pattern_matchs]))
print(np.sum([len(annotations['entities']) for text, annotations in test_augment_text_entities]))

3000
4312
4312


In [214]:
train_data = train_text_entities
train_data_aug = train_text_entities + train_augment_text_entities
test_data = test_text_entities
test_data_aug = test_text_entities + test_augment_text_entities

print(len(train_data))
print(len(train_data_aug))
print(len(test_data))
print(len(test_data_aug))

4582
13582
1528
4528


### Generate NER data

In [215]:
def generate_data(data):
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return db
    
train_spacy = generate_data(train_data_aug)
train_spacy.to_disk("./train_aug.spacy") # save the docbin object
test_spacy = generate_data(test_data) 
test_spacy.to_disk("./test_aug.spacy") # save the docbin object

100%|██████████| 1528/1528 [00:01<00:00, 1242.25it/s]


In [35]:
!ls -l . 

total 7868
-rw-r--r--. 1 root root    1916 Aug 23 01:30 base_config.cfg
-rw-r--r--. 1 root root    1664 Aug 23 09:48 base_config_gpu.cfg
-rw-r--r--. 1 root root    2833 Aug 25 00:26 config.cfg
-rw-r--r--. 1 root root    2625 Aug 25 00:11 config_gpu.cfg
-rw-r--r--. 1 root root  289100 Aug 25 00:31 custom_ner.ipynb
-rw-r--r--. 1 root root  163582 Aug 25 00:33 custom_ner_aug.ipynb
-rw-r--r--. 1 root root  142918 Aug 25 00:29 custom_ner_cpu.ipynb
drwxr-xr-x. 5 root root      43 Aug 24 09:04 output
-rw-r--r--. 1 root root  227413 Aug 25 00:27 test.spacy
-rw-r--r--. 1 root root  229957 Aug 25 00:33 test_aug.spacy
-rw-r--r--. 1 root root  660076 Aug 25 00:27 train.spacy
-rw-r--r--. 1 root root 6312285 Aug 25 00:33 train_aug.spacy


## Train the model

In [36]:
# !python -m spacy train config.cfg --output ./output/cpu --paths.train ./train.spacy --paths.dev ./test.spacy

In [216]:
!python -m spacy train config_gpu_aug.cfg --output ./output/gpu_aug --paths.train ./train_aug.spacy --paths.dev ./test_aug.spacy --gpu-id 0

2021-08-25 06:55:30.030474: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-25 06:55:33,427] [INFO] Set up nlp object from config
[2021-08-25 06:55:33,444] [INFO] Pipeline: ['transformer', 'ner']
[2021-08-25 06:55:33,451] [INFO] Created vocabulary
[2021-08-25 06:55:33,452] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

## Evaluate

###  Check test data

In [217]:
!python -m spacy evaluate output/gpu_aug/model-best ./test_aug.spacy --gpu-id 0

2021-08-25 07:28:54.113892: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   97.04 
NER R   100.00
NER F   98.50 
SPEED   4287  


=============================== NER (per type) ===============================

               P        R       F
PART_NUM   97.04   100.00   98.50



In [218]:
!python -m spacy evaluate output/gpu_aug/model-last ./test_aug.spacy --gpu-id 0

2021-08-25 07:29:20.575229: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   96.57 
NER R   99.90 
NER F   98.21 
SPEED   4344  


=============================== NER (per type) ===============================

               P       R       F
PART_NUM   96.57   99.90   98.21



In [219]:
nlp = spacy.load("output/gpu_aug/model-best") 

In [220]:
options = {'colors': {'ACTION':"#56D7C4", 'PART_NUM':"#92E0AA"} }

def show_text(text, annotations=None, options=options, show_detail=True, standardize_text=None):
    if standardize_text is not None:
        text = standardize_text(text)
    doc = nlp(text)
    print('='*100)
    if show_detail:
        print(doc.text) 
    spacy.displacy.render(doc, style='ent', options=options)
    if show_detail and annotations is not None and len(annotations['entities'])>0:
        print('-'*25, 'acutal entities', '-'*25)
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')   
    if show_detail and len(doc.ents)>0:
        print('-'*25, 'predict entities', '-'*25)
        print(*[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents], sep='\n')      

def show_texts(data, indexes=None, options=options, show_detail=True, standardize_text=None):
    def show_text_(data):
        if isinstance(data, str):
            show_text(data, None, options=options, show_detail=show_detail, standardize_text=standardize_text)
        else:
            text, annotations = data
            show_text(text, annotations, options=options, show_detail=show_detail, standardize_text=standardize_text)  
            
    if indexes is not None:
        for i in indexes:
            show_text_(data[i])   
    else:
        for one_data in data: 
            show_text_(one_data)      
    
                  

In [221]:
indexes = np.random.randint(0, len(test_data), 10)
show_texts(test_data, indexes) 

cu said tuesday is better . i told him i would call tuesday morning to verify afternoon appointment , 07/02 . **please assign to julio hernandez **


/usr/local/lib/python3.6/dist-packages/spacy/displacy/__init__.py:191: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


ordering adf pickup roll ( jc97-04650a ) and separation roll ( jc97-04915a ) . dch . #normal please send this call to doug houseknecht


------------------------- acutal entities -------------------------
('jc97-04650a', 27, 38, 'PART_NUM')
('jc97-04915a', 63, 74, 'PART_NUM')
------------------------- predict entities -------------------------
('jc97-04650a', 27, 38, 'PART_NUM')
('jc97-04915a', 63, 74, 'PART_NUM')
6/29 - dm rich ficek request to assign this to david gibson . thanks 7/1 bad tracking info , escalated to em for clarification . part is in unknown status , unable to pick up . 7/2 still waiting on parts back to me at some point . 500 sheet feeder to south holl


i arrived onsite and spoke with deborah . richard was not in today and she was not sure which system this was for . she let me try to find it out on the floor but i was not able to . richard will be in on tuesday so i will speak to him then . the phone number


...

------------------------- acutal entities -------------------------
('795970-002', 38, 48, 'PART_NUM')
------------------------- predict entities -------------------------
('795970-002', 38, 48, 'PART_NUM')
please order these parts for this machine . # rm2-5758-000cn cassette . #normal please order part number # rm2-2585-000cn . and part # rm2-5752-000cn


------------------------- acutal entities -------------------------
('rm2-5758-000cn', 46, 60, 'PART_NUM')
('rm2-2585-000cn', 107, 121, 'PART_NUM')
('rm2-5752-000cn', 135, 149, 'PART_NUM')
------------------------- predict entities -------------------------
('rm2-5758-000cn', 46, 60, 'PART_NUM')
('rm2-2585-000cn', 107, 121, 'PART_NUM')
('rm2-5752-000cn', 135, 149, 'PART_NUM')
please order part number z7y75-67001 to my hold for pickup in charleston


------------------------- acutal entities -------------------------
('z7y75-67001', 25, 36, 'PART_NUM')
------------------------- predict entities -------------------------
('z7y75-67001', 25, 36, 'PART_NUM')
please assign to jeremy gangl


wrong part was delivered can you please order part no jc97-04907a . please move parts over to new ticket . 


------------------------- acutal entities -------------------------
('jc97-04907a', 54, 65, 'PART_NUM')
------------------------- predict entities -------------------------
('jc97-04907a', 54, 65, 'PART_NUM')
please reassign to francisco noyola . he is aware of reassign . 


In [222]:
show_texts(test_data, indexes, show_detail=False) 

### Check Error Data

In [223]:
def get_error_data(data):
    error_data = []
    for i in range(len(data)):
        text, annotations = data[i]
        doc = nlp(text)
        entities = annotations['entities']
        if len(entities) != len(doc.ents):
            error_data.append(data[i])
            print(i, len(error_data))
        else:
            exit_flag = False
            actual_entities = {(start, end, label): label for start, end, label in entities}
            predict_entities = {(ent.start_char, ent.end_char, ent.label_): ent.label_ 
                                for ent in doc.ents}
            
            for key, _ in actual_entities.items():                
                if key not in predict_entities:
                    print(actual_entities, predict_entities, sep='\n') 
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
            if exit_flag: continue
            for key, _ in predict_entities.items():
                if key not in actual_entities:
                    print(actual_entities, predict_entities)
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
                
    return error_data

error_data = get_error_data(test_data)

89 1
281 2
408 3
462 4
548 5
561 6
573 7
599 8
645 9
691 10
705 11
721 12
785 13
834 14
1006 15
1211 16
1227 17
1303 18
1330 19
1366 20
1415 21
1430 22
1519 23
1526 24


In [224]:
print(len(error_data))

24


In [225]:
show_texts(error_data)  

revisit 7/9 please order dell-snpcrxj6c to san diego hal . please refer part order to multivendor team - dell diagnosis - failed memory diagnostics . waiting for user , but never returned call back . 


------------------------- predict entities -------------------------
('dell-snpcrxj6c', 25, 39, 'PART_NUM')
7/1/21 16:28 jo ; emailed hp mps team to please reassign this wo . the adm jose de sousa has directed us not to service any of these models . per adm , &quot ; if we get any call for the models below it should not go to blm – they are new canon devices and are . 7/1/21 16:28 jo ; emailed hp mps team to please reassign this wo . the adm jose de sousa has directed us not to service any of these models . per adm , &quot ; if we get any call for the models below it should not go to blm – they are new canon devices and are


------------------------- predict entities -------------------------
('– they', 224, 230, 'PART_NUM')
('– they', 487, 493, 'PART_NUM')
please clone . room: f4535-surf permit


------------------------- predict entities -------------------------
('f4535-surf', 21, 31, 'PART_NUM')
1- 1702nk0un0 ( mk-6325 maintenance kit ) shipping address: 364 vance avenue , samoa , ca 95564


------------------------- predict entities -------------------------
('mk-6325', 16, 23, 'PART_NUM')
pls order: l0h24-67901 fuser - 110v -tray 2–x roller kit j8j70-67904 -registration assy rm2-6774-000cn nbd to dflt hal pompano . thanks . 


...

------------------------- predict entities -------------------------
('lex-40x536', 17, 27, 'PART_NUM')
please order jc97-04907 scanner assembly . #normal 7/13 please order jc97-04907 scanner assembly due to blue lines when scanning through the adf , cleaned adf scanners thoroughly and could not see visible scratches on glass , signed for sc kp


------------------------- predict entities -------------------------
('jc97-04907', 13, 23, 'PART_NUM')
('jc97-04907', 69, 79, 'PART_NUM')
please order formatter/f2a68-67915 , emmc/5851-6436 . please ship to my default fedex-hfpu , 1488 w pipeline rd suite 200 , hurst , tx 76053 , attn: hp-ce ken vester . 


------------------------- predict entities -------------------------
('emmc/5851-6436', 37, 51, 'PART_NUM')
please order the following drum ( sam-clt-r806x ) 


------------------------- predict entities -------------------------
('sam-clt', 34, 41, 'PART_NUM')
please order recommended part nbd to wo address under user’s name 7/16- part eta change from 10:30am to 4pm part eta : end of the day ! e-mail user with tracking information . waiting for part 7/19- part status: “potentially delayed” 7/20- please order j


------------------------- predict entities -------------------------
('“potentially', 212, 224, 'PART_NUM')
('” 7/20-', 232, 239, 'PART_NUM')


In [47]:
doc = nlp('rm26322-000cn PART_NUM e6b69-67901 PART_NUM rm1-8413-000cn PART_NUM please send ACTION to fedex ACTION 3242 herrman rd garland , texas 75041 on 2021-08-21')
print(*[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents], sep='\n') 
spacy.displacy.render(doc, style='ent', options=options)

('e6b69-67901', 23, 34, 'PART_NUM')
('rm1-8413-000cn', 44, 58, 'PART_NUM')


### Some Features and Issues

#### Miss some part-numbers 

In [226]:
sample_parts = ['M14302-161', '926537-001', 'M46915-D01', '140314-FB2', '176253-8B0', 
               '442010-ABC', 'L07896-DB1', '168757-B21', 'FE-15520-01', 'RM1-7867-000CN', 
               '5066-4719', '5066-3872-1', '3HZ90-30001', 'T6M22-30001', '98544-04105', 
               'C8109-69018', 'L26480-DB1', '632427-001B', 'CN463-69003', 'RM1-1298BULK', 
               '8-752-078-46', 'X-4035-119-1', 'TBLB3002-A04', '4822-701-15319', '08-2212C-33W', 
               '19-40065-011', 'ATT09850-66532', '98564-66572-SQ', '897-250144AA', '738-576-94', 
               '1FQ6-0001', '1818-7469-LGSA', '1820-6361M', '1818-6868-SAM', '3138-107-96160', 
               '686858R-999', 'MD30C-A2', '29-26196-00', 'DEL-7D092', 'LEX-40X2384', 'LEX-40X2831H', 
               'SAM-6107001172', 'SHA-1625DS51', 'XER-006R01275', 'BRO-TN115BK', 'DEL-310-5811', 
               'DEL-PK496', 'KYO-1702F97US0', 'LA95-CA', 'KYO-DK-310H', 'EPS-C31CD38A9921', 
               'KYO-F994091H', 'EPS-CEPS-003G', 'LEX-24015SA', 'LEX-62D1X00', 'LEX-72K0DV0', 
               'LEX-801HC', 'LEX-C5220CSH', 'TRO-78-24619-001', 'LEX-C544X1CG', 'HPE-749797-001', 
               'SFT-P-895', 'LEX-E260A21A-C', 'RIC-D0296509', 'SEA-ST1000LM049', 'LEX-12A8400-PC', 
               'HPE-JL258A', 'RIC-400507', 'FUJ-FPCPR362AQ', 'TOS-6LE8296100', 'XER-003K04980', 
               'XER-116111500', 'ATS-2711FXSC-901', 'BRO-TN336BK', 'DEL-310-5807', 'HPI-CF258A-M', 
               'LEX-T101-0000000', 'DEL-9PN5P', 'SAM-BA92-08880A', 'APC-RBC7', 'KMH-4448-121', 
               'SXC-C7ABTTAAB', 'TRO-04621201', 'VEF-28924-04-R', 'WFP-HPM527Z', 'KYO-1702LK0UN2', 
               'APC-SUA1500RM2U', 'KYO-302NM18021', 'HPE-JH329-61001', 'KYO-1T02GA0US0', 'LEB-04X4674',
               'KYO-302GR93034', 'DEL-V1RX3', 'LEX-40X0593', 'TOS-6LH3460800', 'RIC-AE020171', 
               'XER-59K60140', 'BRO-LM5140001', 'KMH-A00JA56600', 'KYO-302F906240', 'BRO-LEM084001', 
               'XER-116-2035', 'BRO-BU100CLH', '3X-LK465-A2', '4G1-3999-030CN', 'H3980-60002BULK', 
               'SHA-VHI0MFP000', 'TOS-C017839000', 'JC66-01190A', 'XER-3335DNI', 'XER-3635MFP-S', 
               'XER-4622DN', 'XER-7750-Z2', 'DEL-D1320-W3', 'DEL-P637D', 'LEX-C540X35G', 
               'SHA-CPLTM8190F', 'SHACFRM-1380DS53', 'XER-401-0855-0', '90-0077', 'BRO-LF6710001', 
               'Q3938-68001-PCA', 'C314X+49A-90002', '20ER-84530KC', 'RM2-2903-BULK', 'CE506-67919-BU', 
               'CH971-91596', 'DEL-FM235', 'JC44-00210E', 'SHACFRM-1509DS52', 'SHADHAI-5043FCPZ', 
               'SHAGCAB-1102FCB1', '000-50-02-008', '504000226-DD', 'SHAVHPLG217L5A-1', 'SHAVHPOSH03Y02-1', 
               'SHADUNT-8962FCP1', 'SHAKI-OK0010FCPZ', 'SHAQSW-P0506FCZZ', 'SHAVHPGP1A73AR-1', 'SHARH-IX0055FCPZ', 
               '358-000001', 'SHAVHPGP1A71L3-18', 'B23-0994', '504000226-D', '5066-0731', 
               'E0-CABLE-01', 'RM1-8508-010-BU', 'B3M77-67902-BU', 'CA02626-E029FJ', 'CF065-67901-BU', 
               'C2H57-67901-BULK', 'LN08-A2', '693709-001B', 'SS467-67001', 'LEX-70C0D20', 
               'SHAQSW-M0518FCPZ', 'SHALX-BZ0994FCPZ', 'LN08X-TF', 'SHAGCAB-1507FCAZ']

sample_parts = [part.lower() for part in sample_parts]

In [227]:
def get_texts(parts, standardize_text=standardize_text1):
    texts = [standardize_text(f'Please order qty 1 {part} NBD to my HAL in Lenexa. Thank you.') for part in parts]
    return texts

def check_sample_parts(parts):  
    texts = get_texts(parts)
    docs = nlp.pipe(texts)
    texts = []
    error_texts = []
    errpr_parts = []
    for doc, part in zip(docs, parts):
        texts.append(doc.text)
        if len(doc.ents) !=1 or doc.ents[0].text != part.lower():
            error_texts.append(doc.text)
            errpr_parts.append(part)
    return texts, error_texts, errpr_parts
        
parts = ['000-50-02-008']
texts, error_texts, errpr_parts = check_sample_parts(parts)
show_texts(texts, show_detail=True) 

please order qty 1 000-50-02-008 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('000-50-02-008', 19, 32, 'PART_NUM')


In [229]:
parts = sample_parts
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')
show_texts(error_texts, show_detail=True)  

find 0/160 errors


In [230]:
error_text_entities, error_pattern_matchs = generate_label(error_texts, patterns=patterns)
show_entity_label(error_text_entities, error_pattern_matchs)

('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d[a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('[a-zA-Z][a-zA-Z]-\\d\\d\\d\\d\\d-\\d\\d', 0)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d-\\d', 0)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z]', 0)
('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 0)
('\\d-\\d\\d\\d-\\d\\d\\d-\\d\\d', 0)
('[a-zA-Z]-\\d\\d\\d\\d-\\d\\d\\d-\\d', 0)
('[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\\d

I think that the issues are mainly caused by lacking of these parts in training data. 

There are 2 solutions:

1. merge the results of regular expressions and the model.
2. use data augmentation to generate a lot of random part numbers based on the rules.

####  Find potentail part number

In [236]:
parts = ['FE-15520-01', 'FE-15520-0123']
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')
show_texts(texts, show_detail=True) 

find 0/2 errors
please order qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('fe-15520-01', 19, 30, 'PART_NUM')
please order qty 1 fe-15520-0123 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('fe-15520-0123', 19, 32, 'PART_NUM')


In [235]:
some_text_entities, some_pattern_matchs = generate_label(texts, patterns=patterns)
show_entity_label(some_text_entities, some_pattern_matchs)

--------------------------------------------------
please order qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . 
('fe-15520-01', 19, 30, 'PART_NUM')
--------------------------------------------------
please order qty 1 fe-15520-0123 nbd to my hal in lenexa . thank you . 

('[a-zA-Z][a-zA-Z]-\\d\\d\\d\\d\\d-\\d\\d', 1)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d[a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d-\\d', 0)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('

#### find potential actions 

it has some effects, but not good enough.

In [237]:
texts = ['order qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'orders qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'send qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'sends qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ship qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ships qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . 2012-02-02',         
        ]

show_texts(texts, show_detail=False) 

## Output Results

In [ ]:
parts = 